# FUNCIONES
Este script contiene las funciones necesarias para tomar los datos del usuario y realizar una predicción con el modelo NGBoost

### Librerías

In [83]:
#Carga del modelo
import pickle
import pandas as pd
import numpy as np

#librerías de pre-processing
import datetime as dt

#librería del modelo
from ngboost import NGBRegressor

# Cargado datos

In [84]:
pickle_in = open('../stories/modelo.pickle', 'rb')
#Unpickle modelo
best_model = pickle.load(pickle_in)

#Unpickle temperatura
media_train_temp = pickle.load(pickle_in)
std_train_temp = pickle.load(pickle_in)
ult_temp = pickle.load(pickle_in)

#Unpickle poblacion
media_train_pob = pickle.load(pickle_in)
std_train_pob = pickle.load(pickle_in)
ult_pob = pickle.load(pickle_in)

#Unpickle PIB
media_train_pib = pickle.load(pickle_in)
std_train_pib = pickle.load(pickle_in)
ult_pib = pickle.load(pickle_in)

#Unpickle desempleo
media_train_des = pickle.load(pickle_in)
std_train_des = pickle.load(pickle_in)
ult_des = pickle.load(pickle_in)

#Unpickle rezagos
media_train_rez = pickle.load(pickle_in)
std_train_rez = pickle.load(pickle_in)
ult_rez = pickle.load(pickle_in)

#Unpickle muestra
x_muestra = pickle.load(pickle_in)

#Unpickle festivos
base_festivos = pickle.load(pickle_in)

# Función de predicciones
Recibe un vector de predictores y regresa la media y desviación estandar de la predicción con el mejor modelo entrenado NGBoost

In [85]:
def predecir(x_pred, modelo = best_model):
    y_dist = modelo.pred_dist(x_pred)
    media = y_dist.params['loc'][0]
    std = y_dist.params['scale'][0]
    return(media, std)

# Función user to predictores
Esta función recibe los parámetros escritos por el usuario y los convierte en un vector de predictores apto para el modelo

In [142]:
#Parámetros de prueba:
inc_temp = 20 #grados
inc_pob = 0.00
inc_pib = 0.5
inc_des = 0.00
fecha = '2021-12-24'

In [86]:
def get_predictores(inc_temp, fecha, inc_pib,inc_pob, inc_des, med_temp=media_train_temp, std_temp=std_train_temp, ult_temp=ult_temp, med_pob=media_train_pob, std_pob=std_train_pob, ult_pob=ult_pob, med_pib=media_train_pib, std_pib=std_train_pib, ult_pib=ult_pib,  med_des=media_train_des, std_des=std_train_des, ult_des=ult_des, med_rez=media_train_rez, std_rez=std_train_rez, ult_rez=ult_rez, base_festivos=base_festivos, x_muestra=x_muestra ):
    # Tratamiento de temperaturas
    temp = ult_temp+ inc_temp
    temp_est = (temp - med_temp)/std_temp
    temp_est = temp_est.to_list()
    
    # Población
    poblacion = ult_pob*(1+inc_pob)
    poblacion = (poblacion-med_pob)/std_pob
    
    # Tratamiento del PIB
    pib = ult_pib*(1+inc_pib)
    pib_est = (pib-med_pib)/std_pib
    pib_est = pib_est.to_list()
    
    # Tratamiento del desempleo
    des = ult_des*(1+inc_des)
    des_est = (des-med_des)/std_des
    
    # Rezagos
    rez = (ult_rez-med_rez)/std_rez
    
    list_predictores = [fecha]+[rez] + temp_est + [poblacion] + pib_est + [des_est]
    predictores = pd.DataFrame(list_predictores, index= ['fecha']+['rez'] + pd.DataFrame(ult_temp).T.columns.to_list()+['Población económicamente activa']+pd.DataFrame(ult_pib).T.columns.to_list() + ['tasa_desempleo'] ).T
    
    # Tratamiento de fechas
    predictores['fecha'] = pd.to_datetime(predictores['fecha'], format='%Y-%m-%d')
    
    dias = []
    for i in predictores['fecha']:
        dias.append(pd.Timestamp(i).day_name())
    meses = []
    for i in predictores['fecha']:
        meses.append(pd.Timestamp(i).month_name())
    predictores['dia_semana'] = dias
    predictores['nombre_mes'] = meses
    
    base_festivos['fecha'] = pd.to_datetime(base_festivos['fecha'], format='%d/%m/%Y')
    
    predictores['week_end'] = [1 if x == 'Sunday' or x == 'Saturday' else 0 for x in predictores['dia_semana']]
    predictores = pd.merge(predictores,base_festivos, how='left', left_on='fecha', right_on='fecha',right_index=False)
    predictores.drop(columns=['fecha'], inplace=True)
    predictores['festivo_we']=predictores.apply(lambda x: 1 if x['festivo']==1 or x['week_end']==1 else 0,axis=1)
    
    predictores['August'] = np.where(predictores['nombre_mes']=='August', 1, 0)
    predictores['December'] = np.where(predictores['nombre_mes']=='December', 1, 0)
    predictores['February'] = np.where(predictores['nombre_mes']=='February', 1, 0)
    predictores['January'] = np.where(predictores['nombre_mes']=='January', 1, 0)
    predictores['July'] = np.where(predictores['nombre_mes']=='July', 1, 0)
    predictores['June'] = np.where(predictores['nombre_mes']=='June', 1, 0)
    predictores['March'] = np.where(predictores['nombre_mes']=='March', 1, 0)
    predictores['May'] = np.where(predictores['nombre_mes']=='May', 1, 0)
    predictores['November'] = np.where(predictores['nombre_mes']=='November', 1, 0)
    predictores['October'] = np.where(predictores['nombre_mes']=='October', 1, 0)
    predictores['September'] = np.where(predictores['nombre_mes']=='September', 1, 0)
        
    predictores.drop(columns=['week_end','dia_semana','festivo','nombre_mes'], inplace=True)
    
    orden = pd.DataFrame(x_muestra).T.columns.to_list()
    predictores = predictores[orden]
    
    return(predictores)

# HACIENDO PREDICCIONES

In [88]:
get_predictores(inc_temp, fecha, inc_pib,inc_pob, inc_des).shape

(1, 57)

In [133]:
predecir(get_predictores(inc_temp, fecha, inc_pib,inc_pob, inc_des))

(1.1303449755934847, 0.23028796017489817)

In [143]:
predecir(get_predictores(inc_temp, fecha, inc_pib,inc_pob, inc_des))

(0.8132122425200109, 2.937263704023603)